In [1]:
import json
import pandas as pd


In [2]:
chsd_train = pd.read_csv("train.csv", encoding="utf-8")
chsd_test = pd.read_csv("test.csv", encoding="utf-8")

In [7]:
chsd_texts = []
for i in range(len(chsd_train)):
    chsd_texts.append(chsd_train.iloc[i, 1].strip())
for i in range(len(chsd_test)):
    chsd_texts.append(chsd_test.iloc[i, 1].strip())

In [9]:
COLD_lexicons = []
Toxic_lexicons = []
with open("../COLD/COLD_lexicons.json", "r", encoding="utf-8") as f:
    COLD_lexicons = json.loads("".join(f.readlines()))
with open("../Toxic_rest/full_lexicon.json", "r", encoding="utf-8") as f:
    Toxic_lexicons = json.loads("".join(f.readlines()))

In [10]:
len(chsd_texts)

17430

In [11]:
have_lexicon_COLD = []
have_lexicon_Toxic = []
for i in range(len(chsd_texts)):
    COLD_flag, Toxic_flag = False, False
    for l in COLD_lexicons:
        if l in chsd_texts[i]:
            have_lexicon_COLD.append(1)
            COLD_flag = True
            break
    if not COLD_flag:
        have_lexicon_COLD.append(0)
    for l in Toxic_lexicons:
        if l in chsd_texts[i]:
            have_lexicon_Toxic.append(1)
            Toxic_flag = True
            break
    if not Toxic_flag:
        have_lexicon_Toxic.append(0)

        

In [12]:
sum(have_lexicon_COLD), sum(have_lexicon_Toxic)

(15502, 4492)

In [16]:
COLD_test = []
with open("../COLD/COLD_base.json", "r", encoding="utf-8") as f:
    COLD_test = json.loads("".join(f.readlines()))
COLD_test = [t["text"].strip() for t in COLD_test]
len(COLD_test)

10415

In [17]:
chsd_base = []
for i, text in enumerate(chsd_texts):
    if not text.strip() in COLD_test and (have_lexicon_COLD[i] or have_lexicon_Toxic[i]):
        chsd_base.append(text.strip())

In [18]:
len(chsd_base)

6884

In [22]:
chsd_base_toxic, chsd_base_cold, chsd_base_both = [], [], []
for text in chsd_base:
    f1, f2 = 0, 0
    for l in COLD_lexicons:
        if l in text:
            f1 = 1
            break
    for l in Toxic_lexicons:
        if l in text:
            f2 = 1
            break
    if f1 and f2:
        chsd_base_both.append(text)
    elif f1:
        chsd_base_cold.append(text)
    elif f2:
        chsd_base_toxic.append(text)

In [23]:
len(chsd_base_toxic), len(chsd_base_cold), len(chsd_base_both)

(542, 4636, 1706)

In [24]:
import random

random.seed(2025)
random.shuffle(chsd_base_cold)
chsd_base_cold = chsd_base_cold[:len(chsd_base_toxic)]

In [25]:
chsd_base = chsd_base_toxic + chsd_base_cold + chsd_base_both
len(chsd_base)

2790

In [26]:
chsd_base = [{"text": x} for x in chsd_base]

In [27]:
with open("CHSD_base.json", "x", encoding="utf-8") as f:
    f.write(json.dumps(chsd_base, ensure_ascii=False))